In [1]:
from pyspark import SparkConf
from pyspark import SparkContext
from pyspark.sql import SparkSession
from operator import add
import nltk
import json
from time import time
#nltk.download('punkt')
#nltk.download('averaged_perceptron_tagger')

In [2]:
_tstart_stack = []
def tic():
    _tstart_stack.append(time())
def toc(fmt="Elapsed: %s s"):
    print (fmt % (time() - _tstart_stack.pop()))

In [3]:
gram_groups = dict()
gram_groups['Adjectives'] = ['JJ', 'JJR', 'JJS']
gram_groups['Nouns'] = ['NN', 'NNS', 'NNP', 'NNPS']
gram_groups['Pronouns'] = ['PRP', 'PRP$']
gram_groups['Adverbs'] = ['RB', 'RBR', 'RBS']
gram_groups['Verb'] = ['VB', 'VBG', 'VBD', 'VBN', 'VBP', 'VBZ']

def get_split_body(rdd):
    body = rdd\
    .map(lambda line: json.loads(line[1])['body'].strip().split())\
    
    return body

def check_gram_grp(tag_tuple):
    word, tag = tag_tuple
    for supergroup, subgroups in gram_groups.items():
                if tag in subgroups:
                    return supergroup
    return None

def categorize_words(split_rdd):
    cat_words = split_rdd.flatMap(lambda word: nltk.pos_tag(word))\
    .map(lambda tupl: (check_gram_grp(tupl), 1))\
    .filter(lambda x: x[0] != None)
   
    return cat_words

def final_test(rdd):
    split = get_split_body(rdd)
    categorized = categorize_words(split)
    group_counts = categorized.reduceByKey(add).collect()
    return group_counts

In [4]:
# (8 cores, 16gb per machine) x 5 = 40 cores
# New API
spark_session = SparkSession\
        .builder\
        .master("spark://192.168.2.119:7077") \
        .appName("ScalibilityTest_")\
        .config("spark.executor.memory", "2g")\
        .getOrCreate()
# Old API (RDD)
#        .config("spark.dynamicAllocation.enabled", True)\
#        .config("spark.shuffle.service.enabled", True)\

spark_context = spark_session.sparkContext

In [5]:
#conf = SparkConf()
#conf.setMaster('spark://192.168.2.119:7077')
#conf.setAppName('Strong_Scalibility')
#spark_context = SparkContext(conf=conf)
#spark_context.setLogLevel("DEBUG")

In [6]:
rdd = spark_context.newAPIHadoopFile(
    'hdfs://localhost:9000/user/ubuntu/RC_2008-01',
    'org.apache.hadoop.mapreduce.lib.input.TextInputFormat',
    'org.apache.hadoop.io.LongWritable',
    'org.apache.hadoop.io.Text'
)

In [8]:
rdd.take(1)

[(0,
  '{"retrieved_on":1425820157,"parent_id":"t1_c02s9rv","distinguished":null,"created_utc":"1199145604","score_hidden":false,"subreddit":"reddit.com","score":4,"author_flair_text":null,"author_flair_css_class":null,"body":"Wow, you\'re a buzz-kill.","ups":4,"id":"c02s9s6","archived":true,"downs":0,"edited":false,"subreddit_id":"t5_6","controversiality":0,"name":"t1_c02s9s6","link_id":"t3_648oh","gilded":0,"author":"Haven"}')]

#### RDD  info

In [21]:
n = rdd.count()
print(n)

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.collectAndServe.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 1 in stage 15.0 failed 4 times, most recent failure: Lost task 1.3 in stage 15.0 (TID 75, 192.168.2.214, executor 1): java.net.ConnectException: Call From team18-node2-new/192.168.2.214 to localhost:9000 failed on connection exception: java.net.ConnectException: Connection refused; For more details see:  http://wiki.apache.org/hadoop/ConnectionRefused
	at sun.reflect.GeneratedConstructorAccessor11.newInstance(Unknown Source)
	at sun.reflect.DelegatingConstructorAccessorImpl.newInstance(DelegatingConstructorAccessorImpl.java:45)
	at java.lang.reflect.Constructor.newInstance(Constructor.java:423)
	at org.apache.hadoop.net.NetUtils.wrapWithMessage(NetUtils.java:792)
	at org.apache.hadoop.net.NetUtils.wrapException(NetUtils.java:732)
	at org.apache.hadoop.ipc.Client.call(Client.java:1479)
	at org.apache.hadoop.ipc.

In [ ]:
np = rdd.getNumPartitions()
print(np)

In [ ]:
#rdd2 = rdd.repartition(2)

In [ ]:
#rdd2.count()

In [ ]:
#x = rdd.take(1)[0][1]
#json.loads(x)

#### Create an rdd with half size

In [ ]:
rdd_half = rdd.sample(False, 0.5)

#### Test with full data

In [ ]:
tic()
final_test(rdd)
toc()

#### Test with half data

In [ ]:
tic()
final_test(rdd_half)
toc()